<h1>
<b>Natural language processing course</b>
            
Spring 2024 - Sharif University of Technology

HW1: Text Exploring

name: Amir Mohammad Ezzati<br>
student-id: 402212269
</h1>

# **Libraries**

In [28]:
!pip install rake-nltk gensim -q

In [ ]:
!pip install yake -q

In [29]:
!pip install keybert -q

In [30]:
import pandas as pd
import numpy as np
import random
import tqdm
import nltk, re
from nltk import word_tokenize
import string
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from keybert import KeyBERT
import numpy as np
import json
import itertools
from google.colab import drive

In [31]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/nlp/HW1/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data and Overview**

<div dir='rtl'>
<h4>
داده های این notebook مربوط به اخبار health tech است که از پست های اخیر سایت <a href='https://htn.co.uk/'> Health Tech Newspaper </a> کرال شده است.
</h4>
<h4>
200 پست اخیر این سایت در فایل data.json ذخیره و در تمرین استفاده شده است. همچنین این فایل در فولدر تمرین قرار گرفته است.
</h4>
<br>
<h4>
کلیت کد:

در این نوتبوک 200 کانتنت پست های اخبار health tech را بررسی می کنیم. یک سری مراحل پیش پردازشی روی ان انجام میدهیم و همجنین کلمات و عبارات کلیدی چند کلمه ای آن را توسط دو الگوریتم و یک مدل شبکه عصبی آموزش دیده استخراج میکنیم.
</h4>
</div>


In [44]:
with open(data_path+"data.json", "r") as read_file:
    data = json.load(read_file)

<div dir='rtl'>
<h4>
داده های مربوط به پست ها را از فایل json میخوانیم و در یک dataframe ذخیره میکنیم.
</h4>
</div>


In [45]:
titles, contents = [], []
for x in data:
  if 'content' in x.keys():
    titles.append(x['title'])
    content = x['content'].split('RelatedHTN')[0]
    content = content.split('RelatedHealth')[0]
    content = content.replace('\n', ' ')
    contents.append(content)
titles, contents = np.array(titles).reshape(-1, 1), np.array(contents).reshape(-1, 1)

dataset = pd.DataFrame(data=np.concatenate((titles, contents), axis=1), columns=['title', 'content'])

<div dir='rtl'>
<h4>
تعداد جملات و کلمات هر سمپل را محاسبه و در dataframe ذخیره می کنیم. سپس آماری از تعداد کل کلمات و جملات دیتاست ارائه میکنیم.
</h4>
</div>


In [46]:
numberOfSentenses = [len(sent_tokenize(content)) for content in dataset['content']]
dataset['SentsNum'] = numberOfSentenses

In [47]:
def countWords(content):
  sents = sent_tokenize(content)
  count = 0
  for sent in sents:
    count += len(nltk.tokenize.word_tokenize(sent))

  return count

In [48]:
dataset['WordsNum'] = dataset['content'].apply(countWords)

In [49]:
dataset.head()

,title,content,SentsNum,WordsNum
0,Health tech supplier announces ground-breaking...,"By John Kosobucki, CEO, OX.DH In an era where...",21,603
1,HTN Now: Ideal Health on assessing digital mat...,For a recent HTN Now webinar we were joined b...,29,1339
2,News in brief: anxiety app for children in Glo...,Let’s take a look at some of the health tech ...,23,955
3,Hywel Dda Local University Health Board publis...,Hywel DdaHywel Dda Local University Health Bo...,7,385
4,Interview: “A digital nurse is still a nurse a...,"For a recent interview, we spoke with Jessie ...",54,1961


<div dir='rtl'>
<h4>
همانطور که در خروجی خط کد زیر مشخص است، تعداد جملات موجود در دیتاست 4583 جمله است.
</h4>
</div>

In [50]:
print(f"number of sentenses in the dataset is {dataset['SentsNum'].sum()}")

number of sentenses in the dataset is 4468


<div dir='rtl'>
<h4>
تحلیل بر روی کانتنت پست ها
</h4>
</div>

In [51]:
dataset['content']

0       By John Kosobucki, CEO, OX.DH In an era where...
1       For a recent HTN Now webinar we were joined b...
2       Let’s take a look at some of the health tech ...
3       Hywel DdaHywel Dda Local University Health Bo...
4       For a recent interview, we spoke with Jessie ...
                             ...                        
195     Let’s look into some of the health tech news ...
196     As we approach the end of 2023, we’re taking ...
197     A biomedical engineer at Leeds Teaching Hospi...
198     Eight community pharmacy system suppliers hav...
199     For a recent HTN Now webinar, we welcomed Reb...
Name: content, Length: 200, dtype: object

<div dir='rtl' style='font-weight: bold'>
<h4>
در سلول پایین تابع نرمال سازی جملات نوشته شده است. با توجه به نیاز خود می توانیم با این فانکشن عمل های حذف stopwords، حذف punctuations، فرآیند lemitization و stemming را روی جملات انجام دهیم.
</h4>
</div>

In [52]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# some functions are taken from course github
def normalize_sentence(tokenized_sents, minimum_length=2,
                       stopword_removal=True, lower_case=False, punctuation_removal=True,
                       stem=False, lemmatize=False,
                       stopwords_domain=[]):
    '''
    normalization function
    '''
    normalized_sents = tokenized_sents

    if stopword_removal:
        # Remove stopwords in English and also the given domain stopwords
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_sents=[[word for word in sentence if (word.lower() not in stopwords_domain + stopwords)] for sentence in tokenized_sents ]

    if punctuation_removal:
        # Remove punctuations
        normalized_sents=[[word for word in sentence if word not in string.punctuation] for sentence in normalized_sents ]

    if stem:
      ps = PorterStemmer()
      normalized_sents = [[ps.stem(word) for word in sentence] for sentence in normalized_sents]

    if lemmatize:
      wordnet_lemmatizer = WordNetLemmatizer()
      normalized_sents = [[wordnet_lemmatizer(word) for word in sentence] for sentence in normalized_sents]

    if lower_case:
        # Convert everything to lowercase and filter based on a min length
        normalized_sents=[[word.lower() for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    elif minimum_length>1:
        normalized_sents= [[word for word in sentence if len(word)>minimum_length] for sentence in normalized_sents ]

    return normalized_sents

<div dir='rtl' style='font-weight: bold'>
<h4>
در سلول پایین کانتنت های موجود در دیتاست که هرکدام متشکل از چندین جمله هستند را tokenize میکنیم تا به جملاتش دسترسی داشته  باشیم.
</h4>
</div>

In [53]:
def getSentences(data):
  sentences = []
  for content in data:
    sents = sent_tokenize(content)
    sentences.extend(sents)

  return sentences

sentences = getSentences(dataset['content'])

<div dir='rtl' style='font-weight: bold'>
<h4>
بخش پایین شامل word tokenization است که با استفاده از چند عبارت Regex انجام می شود.        

ابتدا قرار است بدون انجام فرآیند lemitization و stemming کلمات پرتکرار موجود در دیتاست را لیست کنیم.
</h4>
</div>

In [54]:
pattern = r'''(?x)          # set flag to allow verbose regexps
        (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
      | \w+(?:-\w+)*        # words with optional internal hyphens
      | \$?\d+(?:\.\d+)?%?\s?  # currency and percentages, e.g. $12.40, 82%
      | \.\.\.              # ellipsis
      | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
    '''
pattern = re.compile(pattern)

def getTokenizedSentence(sentences):
  tokenized_sentences = []
  for sentence in sentences:
    tokenized = nltk.regexp_tokenize(sentence, pattern)
    tokenized_sentences.append(tokenized)

  return tokenized_sentences

# word tokenizing
tokenized_sentences = getTokenizedSentence(sentences)
# normalizing
normalized_sentences = normalize_sentence(tokenized_sentences)

In [55]:
normalized_sentences[1]

['pioneering',
 'UK-based',
 'healthcare',
 'technology',
 'company',
 'links',
 'University',
 'Oxford',
 'mission',
 'digitally',
 'transform',
 'healthcare',
 'modern',
 'intuitive',
 '100',
 'cloud-native',
 'solutions',
 'built',
 'secure',
 'industry-leading',
 'Microsoft',
 'Azure',
 'cloud',
 'platform']

In [56]:
sentence_words = list(itertools.chain(*normalized_sentences))
print ('%-16s' % 'Number of words', '%-16s' % len(sentence_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(sentence_words)))
avg=np.sum([len(word) for word in sentence_words])/len(sentence_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
avg_sentence_length_c=np.mean([len(' '.join(sentence)) for sentence in sentence_words])
print ('%-16s' % 'Average sentence length in characters', '%-16s' % avg_sentence_length_c)
print ('%-16s' % 'Longest word', '%-16s' % sentence_words[np.argmax([len(word) for word in sentence_words])])

Number of words  88252           
Number of unique words 9510            
Average word length 6.8887730589675025
Average sentence length in characters 12.777546117935005
Longest word     internationally-recognised


In [57]:
freqdist = FreqDist(itertools.chain(*normalized_sentences))
top20words = freqdist.most_common(10)

print ('%-16s' % 'word', '%-16s' % 'count','%-16s' %  '% of the total sentences')
for topword in top20words:
    percent=(topword[1]/len(normalized_sentences))*100
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

word             count            % of the total sentences
digital          1106             24.7538048343778
care             888              19.874664279319607
NHS              876              19.606087735004476
data             694              15.532676812891674
patients         620              13.876454789615039
patient          574              12.846911369740377
health           564              12.623097582811102
support          470              10.519247985675918
also             388              8.683974932855863
people           385              8.616830796777082


<div dir='rtl' style='font-weight: bold'>
<h4>
اینبار بعد از انجام فرآیند stemming کلمات پرتکرار را استخراج میکنیم.
</h4>
</div>

In [58]:
normalized_sentences = normalize_sentence(tokenized_sentences, stem=True)

In [59]:
sentence_words = list(itertools.chain(*normalized_sentences))
print ('%-16s' % 'Number of words', '%-16s' % len(sentence_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(sentence_words)))
avg=np.sum([len(word) for word in sentence_words])/len(sentence_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
avg_sentence_length_c=np.mean([len(' '.join(sentence)) for sentence in sentence_words])
print ('%-16s' % 'Average sentence length in characters', '%-16s' % avg_sentence_length_c)
print ('%-16s' % 'Longest word', '%-16s' % sentence_words[np.argmax([len(word) for word in sentence_words])])

Number of words  86952           
Number of unique words 5738            
Average word length 5.708850860244732
Average sentence length in characters 10.417701720489465
Longest word     ospedaliero-universitaria


In [60]:
freqdist = FreqDist(itertools.chain(*normalized_sentences))
top20words = freqdist.most_common(10)

print ('%-16s' % 'word', '%-16s' % 'count','%-16s' %  '% of the total sentences')
for topword in top20words:
    percent=(topword[1]/len(normalized_sentences))*100
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

word             count            % of the total sentences
digit            1285             28.76007162041182
patient          1231             27.55147717099373
care             1041             23.29901521933751
health           837              18.733213965980305
data             764              17.099373321396598
trust            641              14.346463742166518
support          639              14.301700984780663
servic           634              14.189794091316024
use              630              14.100268576544314
work             604              13.5183527305282


<div dir='rtl' style='font-weight: bold'>
<h4>
نتیجه:   

همانطور که پیداست، تعداد کلمات unique موجود در دیتاست در این حالت کاهش زیادی داشته است. چون در فرآیند stemming پسوند اضافه کلمات حذف شده و ریشه کلمات باقی میمانند. به همین دلیل تعداد کلمات unique کاهش میابد.


یکی از مزیت های stem کردن برای یافتن کلمات پرتکرار این است که فاقد از اینکه کلمه اسم یا صفت بوده می توانیم میزان تکرار ریشه آن را متوجه شویم. همچنین افعال پرتکرار فاقد از اینکه در چه گرامری استفاده شده باشد را میتوانیم مقایسه کنیم.


اگر دقیق تر نگاه کنیم میبینیم که کلمه patient که هم بصورت مفرد و هم جمع در بخش قبلی وجود داشت، الان به یک صورت(ریشه) یعنی patient  در این قسمت آمده است.

</h4>
</div>

# **Extracting Keyphrases**

# **Rake**
<div dir='rtl' style='font-weight: bold'>
<h4>
<b>
در خط های بعدی به ترتیب بیشترین عبارت های کلیدی 2 کلمه ای تا 5 کلمه ای را با استفاده از کتابخانه rake_nltk استخراج می کنیم.
</b>
</h4>
</div>

In [33]:
from rake_nltk import Rake

def mostCommonPhrases(dataset, length=2):
  r = Rake(min_length=length, max_length=length, include_repeated_phrases=False)

  phrases_count = {}
  for content in dataset['content']:

    r.extract_keywords_from_text(content)
    keywords_rake = r.get_ranked_phrases()

    for keyword in keywords_rake:
      if keyword in phrases_count.keys():
        phrases_count[keyword] += 1
      else:
        phrases_count[keyword] = 1

  top10Keys = sorted(zip(phrases_count.values(), phrases_count.keys()), key=lambda x: x[0], reverse=True)[:10]
  for key in top10Keys:
    print(f'{key[1]}:  {key[0]}')

In [34]:
mostCommonPhrases(dataset, length=2)

nhs england:  49
trust ’:  41
please click:  40
primary care:  29
digital health:  26
rewired 2024:  23
social care:  22
last year:  22
last month:  21
patient ’:  21


In [35]:
mostCommonPhrases(dataset, length=3)

digital access e:  12
managing cyber threats:  11
health innovation network:  11
nhs foundation trust:  10
prior information notice:  10
tackling ai risks:  9
hett north 2024:  9
federated data platform:  9
nhs england ’:  8
driving patient outcomes:  8


In [36]:
mostCommonPhrases(dataset, length=4)

chief clinical information officer:  12
relatedgoogle cloud announces updates:  12
huddersfield nhs foundation trust:  8
digital transformation … htn:  5
powys teaching health board:  5
deputy chief medical officer:  5
digital technology assessment criteria:  4
north cumbria ics pledges:  4
help reduce … htn:  4
nhs england clinical entrepreneur:  4


In [37]:
mostCommonPhrases(dataset, length=5)

south essex nhs foundation trust:  5
25 gp contract places focus:  5
south yorkshire primary care workforce:  5
chief data officer albert king:  5
primary care access recovery plan:  5
leeds teaching hospitals nhs trust:  4
heart "… new systems introduced:  4
humber teaching nhs foundation trust:  4
support electronic patient record delivery:  4
south yorkshire digital health hub:  3


<div dir='rtl' style='font-weight: bold'>
<h4>
نتیجه:   

همانطور که مشخص است عبارات کلیدی پر استفاده در دیتاست با طول های مختلف لیست شده است.   

برخی از عبارات استخراج شده عبارت درست و معنی داری  نیست یا یکی از کلماتش شامل نمادهای(" و ' و - و ..) است.      

این موضوع میتواند بدلیل دیتاست کثیف یا پیش پردازش ناکامل یا الگوریتم استخراج ضعیف بوجود بیاید.
</h4>
</div>

# **Yake**
<div dir='rtl' style='font-weight: bold'>
<h4>
<b>
در خط های بعدی به ترتیب بیشترین عبارت های کلیدی 2 کلمه ای و 3 کلمه ای را با استفاده از کتابخانه yake استخراج می کنیم.
</b>
</h4>
</div>

In [76]:
# python snippet for yake
from yake import KeywordExtractor as Yake

def mostCommonPhrases(dataset, length=2):
  yake = Yake(lan="en", stopwords='english')

  phrases_count = {}
  for content in dataset['content']:
    yake_keyphrases = yake.extract_keywords(content)
    yake_keyphrases = list(dict(yake_keyphrases).keys())

    for keyword in yake_keyphrases:
      if keyword in phrases_count.keys():
        phrases_count[keyword] += 1
      else:
        if len(keyword.split(' ')) == length:
          phrases_count[keyword] = 1

  top10Keys = sorted(zip(phrases_count.values(), phrases_count.keys()), key=lambda x: x[0], reverse=True)[:10]
  for key in top10Keys:
    print(f'{key[1]}:  {key[0]}')

In [77]:
mostCommonPhrases(dataset, length=2)

NHS Foundation:  33
Foundation Trust:  32
NHS England:  26
the NHS:  25
the trust:  16
NHS Trust:  10
and the:  9
with the:  9
for the:  9
Hospitals NHS:  8


In [69]:
mostCommonPhrases(dataset, length=3)

NHS Foundation Trust:  57
NHS England has:  15
Foundation Trust has:  14
Hospitals NHS Trust:  12
Health and Care:  10
Hospitals NHS Foundation:  10
Hospital NHS Foundation:  9
University Hospitals NHS:  7
and Care Wales:  7
Healthcare NHS Trust:  6


<div dir='rtl' style='font-weight: bold'>
<h4>
نتیجه:   

برخی از عبارات استخراج شده عبارت درست و معنی داری  نیست. مثلا عبارتی که دارای کلمه and است یا فعل has دارد.      

این موضوع میتواند بدلیل نداشتن قابلیت حذف stopwords  یا الگوریتم استخراج ضعیف بوجود بیاید.

بطور کلی الگوریتم rake کارایی بیشتر داشت و عبارات کلیدی طولانی تر از سه کلمه را نیز استخراج کرد.
</h4>
</div>

# **KeyBert**
<div dir='rtl' style='font-weight: bold'>
<h4>
<b>
در خط های بعدی به ترتیب بیشترین عبارت های کلیدی را با استفاده از مدل KeyBert استخراج می کنیم.
</b>
</h4>
</div>

In [38]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [79]:
keywords = kw_model.extract_keywords(dataset['content'][0],
                                     keyphrase_ngram_range=(1, 4),
                                     stop_words='english',
                                     highlight=False,
                                     top_n=10)

keywords_list= list(dict(keywords).keys())

keywords

[('platform integrating nhs', 0.7321),
 ('patient centric healthcare ox', 0.7139),
 ('nhs tech innovation', 0.6865),
 ('leveraging latest nhs api', 0.6761),
 ('leveraging latest nhs', 0.6733),
 ('nhs api integrations', 0.6722),
 ('centric healthcare ox', 0.6666),
 ('latest nhs api integrations', 0.6643),
 ('nhs tech innovation framework', 0.659),
 ('centric healthcare ox dh', 0.6582)]

In [80]:
keywords = kw_model.extract_keywords(dataset['content'][1],
                                     keyphrase_ngram_range=(1, 4),
                                     stop_words='english',
                                     highlight=False,
                                     top_n=10)

keywords_list= list(dict(keywords).keys())

keywords

[('digital maturity regards medical', 0.7369),
 ('transformation lifecycle digital maturity', 0.7103),
 ('digital maturity assessment roadmap', 0.7023),
 ('digital maturity building roadmaps', 0.6962),
 ('building digital maturity roadmaps', 0.694),
 ('relating digital maturity building', 0.6877),
 ('digital maturity roadmaps', 0.6871),
 ('digital maturity building', 0.6772),
 ('lifecycle digital maturity', 0.677),
 ('digital maturity roadmaps begin', 0.6713)]

In [83]:
def mostCommonPhrases(dataset):
  phrases_count = {}
  for content in dataset['content'][:20]:
    keywords = kw_model.extract_keywords(content,
                                     keyphrase_ngram_range=(1, 4),
                                     stop_words='english',
                                     highlight=False,
                                     top_n=10)
    keywords_list = list(dict(keywords).keys())

    for keyword in keywords_list:
      if keyword in phrases_count.keys():
        phrases_count[keyword] += 1
      else:
        phrases_count[keyword] = 1

  top10Keys = sorted(zip(phrases_count.values(), phrases_count.keys()), key=lambda x: x[0], reverse=True)[:10]
  for key in top10Keys:
    print(f'{key[1]}:  {key[0]}')

In [84]:
mostCommonPhrases(dataset)

nhs app:  2
digital health care wales:  2
electronic health record expanse:  2
platform integrating nhs:  1
patient centric healthcare ox:  1
nhs tech innovation:  1
leveraging latest nhs api:  1
leveraging latest nhs:  1
nhs api integrations:  1
centric healthcare ox:  1


<div dir='rtl' style='font-weight: bold'>
<h4>
نتیجه:   

با استفاده از مدل KeyBert عبارات کلیدی بهتری استخراج شده است. یکی از دلایل این است که مدل KeyBert کلمات stopwords انگلیسی را تشخیص داده و بعنوان عبارت کلیدی در  نظر نمی گیرد.

بدلیل اینکه اجرای مدل KeyBert برای روی هر کانتنت زمان زیادی می برد، از این سو امکان اجرا روی کل دیتاست نبوده تا پرتکرارترین کلمات کلیدی دیتاست را لیست نمایش دهم. اما این کار را روی 20 سمپل از دیتاست انجام دادم و در سلول بالا نتیجه اش مشخص است.
</h4>
</div>